# 第3章 入力評価——分類

 - [一、環境設定](#一、環境設定)
 - [二、ユーザー指示の分類](#二、ユーザー指示の分類)

本章では、入力評価タスクに焦点を当てます。これは、システムの品質と安全性を確保するために極めて重要です。

異なる状況下で多くの独立した指示セットを処理する必要があるタスクでは、まずクエリタイプを分類し、それを基にどの指示を使用するかを決定することに多くの利点があります。

これは、固定カテゴリを定義し、与えられたカテゴリのタスクを処理するのに関連する指示をハードコーディングすることで実現できます。

例えば、カスタマーサービスアシスタントを構築する際、まずクエリタイプを分類し、その分類に基づいてどの指示を使用するかを決定することが非常に重要になる場合があります。

具体例として、ユーザーがアカウントの閉鎖を要求する場合、二次指示にはアカウント閉鎖方法に関する追加説明を加える可能性があります。一方、ユーザーが特定製品の情報を問い合わせる場合、二次指示にはより多くの製品情報を加える可能性があります。

## 一、環境設定
前章と同様に、まずOpenAI APIを使用する環境を設定する必要があります。

In [9]:
import openai
# 导入第三方库

openai.api_key  = "sk-..."
# 设置 API_KEY, 请替换成您自己的 API_KEY

# 以下为基于环境变量的配置方法示例，这样更加安全。仅供参考，后续将不再涉及。
# import openai
# import os
# OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
# openai.api_key = OPENAI_API_KEY

In [2]:
def get_completion_from_messages(messages, 
                                model="gpt-3.5-turbo", 
                                temperature=0, 
                                max_tokens=500):
    '''
    封装一个访问 OpenAI GPT3.5 的函数

    参数: 
    messages: 这是一个消息列表，每个消息都是一个字典，包含 role(角色）和 content(内容)。角色可以是'system'、'user' 或 'assistant’，内容是角色的消息。
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    temperature: 这决定模型输出的随机程度，默认为0，表示输出将非常确定。增加温度会使输出更随机。
    max_tokens: 这决定模型输出的最大的 token 数。
    '''
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # 这决定模型输出的随机程度
        max_tokens=max_tokens, # 这决定模型输出的最大的 token 数
    )
    return response.choices[0].message["content"]

## 二、ユーザー指示の分類

ここでは、システムメッセージ（system_message）をシステムのグローバルガイダンスとして使用し、`#`を区切り文字として選択しています。

区切り文字は、指示や出力内の異なる部分を区別するためのツールで、モデルが各部分を識別するのに役立ち、システムが特定のタスクを実行する際の精度と効率を向上させます。

この例では、`#`を区切り文字として使用することを選択しています。

`#`は独立したトークンと見なすことができるため、理想的な区切り文字です。

In [3]:
delimiter = "####"

这是我们的 system message，我们正在以下面的方式询问模型。

In [4]:
system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Classify each query into a primary category \
and a secondary category. 
Provide your output in json format with the \
keys: primary and secondary.

Primary categories: Billing, Technical Support, \
Account Management, or General Inquiry.

Billing secondary categories:
Unsubscribe or upgrade
Add a payment method
Explanation for charge
Dispute a charge

Technical Support secondary categories:
General troubleshooting
Device compatibility
Software updates

Account Management secondary categories:
Password reset
Update personal information
Close account
Account security

General Inquiry secondary categories:
Product information
Pricing
Feedback
Speak to a human

"""

In [ ]:
system_message = f"""
あなたは顧客サービスクエリを受け取ります。
各顧客サービスクエリは{delimiter}文字で区切られています。
各クエリを主要カテゴリと副次カテゴリに分類してください。
primary および secondary キーを含むJSON形式で出力を提供してください。

主要カテゴリ：請求（Billing）、技術サポート（Technical Support）、アカウント管理（Account Management）、または一般問い合わせ（General Inquiry）。

請求の副次カテゴリ：
購読解除またはアップグレード（Unsubscribe or upgrade）
支払い方法の追加（Add a payment method）
料金の説明（Explanation for charge）
料金の異議申し立て（Dispute a charge）

技術サポートの副次カテゴリ：
一般的なトラブルシューティング（General troubleshooting）
デバイス互換性（Device compatibility）
ソフトウェア更新（Software updates）

アカウント管理の副次カテゴリ：
パスワードリセット（Password reset）
個人情報の更新（Update personal information）
アカウント閉鎖（Close account）
アカウントセキュリティ（Account security）

一般問い合わせの副次カテゴリ：
製品情報（Product information）
価格設定（Pricing）
フィードバック（Feedback）
人間との対話（Speak to a human）

"""

现在我们来看一个用户消息（user message）的例子。

In [26]:
user_message = f"""\ 
I want you to delete my profile and all of my user data"""

In [ ]:
user_message = f"""\ 
私のプロフィールとすべてのユーザーデータを削除してください。"""

このメッセージをメッセージリストとしてフォーマットし、システムメッセージとユーザーメッセージを「####」で区切ります。

人間として考えてみましょう。この文「私にプロフィールを削除してもらいたい」はどのカテゴリに属するでしょうか？

この文は「アカウント管理」、または「アカウント閉鎖」に属するように見えます。

In [7]:
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
]

让我们看看模型是如何思考的

模型的分类是将"Account Management"作为"primary"，"Close account"作为"secondary"。

请求结构化输出（如 JSON）的好处是，您可以轻松地将其读入某个对象中，例如 Python 中的字典。如果您使用其他语言，也可以转换为其他对象，然后输入到后续步骤中。

In [10]:
response = get_completion_from_messages(messages)
print(response)

{
    "primary": "账户管理",
    "secondary": "关闭账户"
}


这是另一个用户消息: "告诉我更多关于你们的平板电视的信息"

我们运用相同的消息列表来获取模型的响应，然后打印出来。

这里返回了另一个分类结果，并且看起来应该是正确的。

In [31]:
user_message = f"""\
Tell me more about your flat screen tvs"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

{
  "primary": "General Inquiry",
  "secondary": "Product information"
}


In [12]:
user_message = f"""\
告诉我更多有关你们的平板电脑的信息"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

以下是针对平板电脑的一般咨询：

{
  "primary": "General Inquiry",
  "secondary": "Product information"
}

如果您有任何特定的问题或需要更详细的信息，请告诉我，我会尽力回答。


因此，根据客户咨询的分类，我们现在可以提供一套更具体的指令来处理后续步骤。

在这种情况下，我们可能会添加关于电视的额外信息，而在其他情况下，我们可能希望提供关闭账户的链接或类似的内容。

在接下来的章节中，我们将进一步了解处理输入的不同方法

在下一章中，我们将探讨更多关于评估输入的方法，特别是如何确保用户以负责任的方式使用系统。